In [1]:
import os
import sys

home_dir = os.path.expanduser("~")

sys.path.append(os.path.join(home_dir, 'Documents/indystudy/icken-and-chegg/'))

from lib import general_utils

In [2]:
from PIL import Image
import PIL.ImageDraw as ImageDraw
import numpy as np
import time

indystudy_path = '/home/sean/Documents/indystudy/'

sys.path.append(indystudy_path)

from keras_yolo3.yolo import YOLO

annotation_path = os.path.join(indystudy_path, 'icken-and-chegg', 'Sean', 'train.txt')
with open(annotation_path) as f:
    lines = f.readlines()

window_size = (416, 416)

detector = YOLO(model_path='/home/sean/Documents/indystudy/output/keras_yolo3/baseline_416_09282018/trained_weights_final.h5',
                anchors_path='/home/sean/Documents/indystudy/keras_yolo3/model_data/yolo_anchors.txt',
                classes_path='/home/sean/Documents/indystudy/keras_yolo3/model_data/custom_classes.txt',
                model_image_size=window_size)

#for line in lines:
line = lines[0]
line = line.split()
image = Image.open(line[0])
iw, ih = image.size
h, w = window_size
gt_boxes = np.array([np.array(list(map(int,box.split(',')))) for box in line[1:]])

draw = ImageDraw.Draw(image)

/home/sean/.virtualenvs/ml/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


/home/sean/Documents/indystudy/output/keras_yolo3/baseline_416_09282018/trained_weights_final.h5 model, anchors, and classes loaded.


In [3]:
predicted_boxes = []
for y in range(0, ih, h):
    for x in range(0, iw, w):
        area = (x, y, x+w, y+h)
        window = image.crop(area)

        # predicted bounding boxes
        _, window_predictions = detector.detect_image_silent(window)

        for bb in window_predictions:
            adjusted_bb = [bb['left'] + x, bb['top'] + y, bb['right'] + x, bb['bottom'] + y]
            
            predicted_boxes.append(adjusted_bb)
            
            draw.rectangle(adjusted_bb, outline="blue")

(416, 416, 3)
Found 0 boxes for img
(416, 416, 3)
Found 0 boxes for img
(416, 416, 3)
Found 0 boxes for img
(416, 416, 3)
Found 0 boxes for img
(416, 416, 3)
Found 0 boxes for img
(416, 416, 3)
Found 0 boxes for img
(416, 416, 3)
Found 0 boxes for img
(416, 416, 3)
Found 0 boxes for img
(416, 416, 3)
Found 0 boxes for img
(416, 416, 3)
Found 0 boxes for img
(416, 416, 3)
Found 0 boxes for img
(416, 416, 3)
Found 0 boxes for img
(416, 416, 3)
Found 0 boxes for img
(416, 416, 3)
Found 0 boxes for img
(416, 416, 3)


KeyboardInterrupt: 

In [5]:
# true bounding boxes
false_negatives = 0
false_positives = 0
true_positives = 0
chopped_detections = 0

used_prediction_indices = {}
for bb in gt_boxes:
    
    preds_for_gt = []
    
    for i in range(len(predicted_boxes)):
        if not used_prediction_indices.get(i, False) and general_utils.box_contains_centroid(bb, utils.get_centroid_of_box(predicted_boxes[i])):
            preds_for_gt.append(predicted_boxes[i])
            
            # mark prediction as used
            used_prediction_indices[i] = True
            
    if len(preds_for_gt) == 0: # nothing detected for gt
        false_negatives += 1

        draw.rectangle([bb[0], bb[1], bb[2], bb[3]], outline="red")
    elif len(preds_for_gt) > 1: # multiple boxes for one gt
        chopped_detections += 1

        draw.rectangle([bb[0], bb[1], bb[2], bb[3]], outline="yellow")
    else:
        if utils.bb_iou([bb[0], bb[1], bb[2], bb[3]], preds_for_gt[0]) > 0.5:
            true_positives += 1
            draw.rectangle([bb[0], bb[1], bb[2], bb[3]], outline="lime")
        else:
            false_negatives += 1
            draw.rectangle([bb[0], bb[1], bb[2], bb[3]], outline="teal")

false_positives = len([i for i in range(len(predicted_boxes)) if not used_prediction_indices.get(i, False)])

precision = true_positives / (true_positives+false_positives)
recall = true_positives / (true_positives+false_negatives)

print(" False Negatives:", false_negatives, "\n",
      "Chopped Detections:", chopped_detections, "\n",
      "True Positives:", true_positives, "\n",
      "False Positives:", false_positives, "\n",
      "Precision:", precision, "\n",
      "Recall:", recall)
    
image.save('first_train_example_detections_with_gt.tiff')
#break

ZeroDivisionError: division by zero

In [4]:
gt_boxes

array([[1243,  509, 1280,  544,    0],
       [ 329,  564,  366,  663,    0],
       [ 331,  686,  368,  792,    0],
       ...,
       [ 839, 6249,  907, 6271,    0],
       [ 910, 6249, 1000, 6271,    0],
       [1002, 6247, 1113, 6272,    0]])

In [16]:
predicted_boxes

[]